In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('./datasets/data_n20_N200.csv',sep=',',header=None)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
0,NaN,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20
1,1.0,2,3,4,7,5,10,6,1,11,...,8,13,17,20,15,18,9,14,16,19
2,2.0,2,1,5,4,6,7,10,8,11,...,13,15,14,18,9,17,3,20,19,16
3,3.0,5,11,4,6,14,12,15,20,9,...,7,13,18,16,1,2,17,3,19,8
4,4.0,16,3,6,8,12,1,19,2,15,...,11,18,9,20,17,10,4,5,13,7


In [13]:
data_arr = np.array(df.loc[1:,1:],dtype=int)
#data_arr = np.concatenate((data_arr,data_arr))
#data_arr = np.concatenate((data_arr,data_arr))
#data_arr = np.concatenate((data_arr,data_arr))
#data_arr = np.concatenate((data_arr,data_arr))
#data_arr = np.concatenate((data_arr,data_arr))
data_arr.shape

(200, 20)

In [14]:
def generate_V(centre):
    centre_arr = np.array(centre)
    n = len(centre_arr)
    V_order = np.zeros(n)
    distToCentre = V_order
    distToCentre = centre_arr - (np.max(centre_arr)+1)/2
    if(n%2):
        V_order[distToCentre==0] = 1
    for i in distToCentre[distToCentre>0]:
        if (np.random.rand()>0.5):
            V_order[distToCentre == i] = abs(i)*2+1
            V_order[distToCentre == -i] = abs(i)*2
        else:
            V_order[distToCentre == i] = abs(i)*2
            V_order[distToCentre == -i] = abs(i)*2+1
            
    return V_order

In [15]:
def rank(ls):
    arr = np.array(ls)
    ordering = np.argsort(arr)
    ranks = np.empty_like(ordering)
    ranks[ordering] = np.arange(len(arr))+1
    return ranks

In [16]:
def pseudo_likelihood(data,n_samples,alpha):
    data_arr = np.array(data)
    N = data_arr.shape[0]
    n = data_arr.shape[1]
    rho_samples = np.zeros((n_samples,n),dtype=int)
    data_rank = rank(np.sum(data_arr,axis=0))
    for i in range(n_samples):
        if((i+1)%100==0):
            print(i+1 ,"/", n_samples, "iterations")
        support = np.arange(0,n)
        rho_tmp = np.zeros(n, dtype = int)
        V_centre = generate_V(data_rank)
        for j in range(n):
            i_curr = np.where(V_centre ==(j+1))
            ####calculating the distance from the data to the possible value
            dists = np.array(list(map(lambda x: np.sum(abs(data_arr[:,i_curr].ravel() - x)),(support+1))))
            log_num =(-alpha/(n)*(dists)) - np.max(-alpha/(n)*(dists))  #minus the max to stablize the exponential term
            log_denom = np.log(np.sum(np.exp(log_num)))
            ###to be very sure that it sums to one, therefore devide by its sum
            probs= np.exp((log_num-log_denom)) / np.sum(np.exp((log_num-log_denom))) 
            index = int(np.where(np.random.multinomial(n=1, pvals = probs)==1)[0])
            rho_tmp[i_curr] = support[index]
            support = support[support!=rho_tmp[i_curr]]
       
        rho_samples[i,:] = rho_tmp+1
           
    return rho_samples


In [17]:
import time
start = time.time()
rho_samples=pseudo_likelihood(data_arr,2000,1)
end = time.time()
#print(end-start)
print(np.round(end - start,3),"seconds elapsed")

100 / 2000 iterations
200 / 2000 iterations
300 / 2000 iterations
400 / 2000 iterations
500 / 2000 iterations
600 / 2000 iterations
700 / 2000 iterations
800 / 2000 iterations
900 / 2000 iterations
1000 / 2000 iterations
1100 / 2000 iterations
1200 / 2000 iterations
1300 / 2000 iterations
1400 / 2000 iterations
1500 / 2000 iterations
1600 / 2000 iterations
1700 / 2000 iterations
1800 / 2000 iterations
1900 / 2000 iterations
2000 / 2000 iterations
6.014 seconds elapsed


In [68]:
rho_samples

array([[ 2,  1,  5, ..., 19, 16, 20],
       [ 3,  2,  5, ..., 20, 17,  1],
       [ 2,  1,  4, ..., 19, 16, 20],
       ...,
       [ 2,  1,  4, ..., 19, 16, 20],
       [ 2,  1,  4, ..., 19, 16, 20],
       [ 2,  1,  4, ..., 19, 16, 20]])